In [1]:
import tensorflow as tf 
import numpy as np 
import tensorflow_probability as tfp
import numpy as np


with open('data.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [2]:
vocab = sorted(set(text))
vocab_size = len(vocab)
n_tokens = len(text)

print(vocab)
print(vocab_size)
print(n_tokens)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65
1115393


In [3]:
text[:80]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.'

In [4]:
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens = vocab_size)
text_vec_layer.adapt(text)
encoded_text = text_vec_layer([text])[0]


In [5]:
n = int(len(text_vec_layer.get_vocabulary())*0.9)
raw_train_set = encoded_text[:n]
raw_test_set = encoded_text[n:]

In [6]:
def to_dataset(sequence, length, shuffle = False, seed = None, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100000, seed = seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [7]:
train_set = to_dataset(raw_train_set, shuffle = True, length = 8)
test_set = to_dataset(raw_test_set, length = 8)

In [100]:
listset = list(train_set)

test = tf.keras.random.randint(shape = (32, 2, 1), minval = 0, maxval = 5)
print(test[:, 0, :])

testpred = tf.random.normal(shape = (32, 8, vocab_size))
testpred = testpred
print(testpred[:, 0, :].shape)
# print(f'when inputs are: {listset[0][0][:5]}')
# print(f'outputs are:{listset[0][1][:5]}')

# print(f'sequence {listset[0][0][0]}')
# print(f'to predict token {listset[0][1][0][-1]}')

print(test[:, 0, :].shape)
ce = tf.keras.losses.SparseCategoricalCrossentropy()
loss = ce(test[:, 0, :], testpred[:, 0, :])
print(loss)


tf.Tensor(
[[3]
 [1]
 [1]
 [4]
 [2]
 [4]
 [4]
 [1]
 [1]
 [2]
 [4]
 [2]
 [4]
 [2]
 [3]
 [2]
 [4]
 [4]
 [0]
 [3]
 [0]
 [4]
 [3]
 [4]
 [2]
 [4]
 [0]
 [4]
 [4]
 [3]
 [1]
 [4]], shape=(32, 1), dtype=int32)
(32, 65)
(32, 1)
tf.Tensor(9.568363, shape=(), dtype=float32)


In [5]:
import tensorflow as tf
import numpy as np 

class AttentionHead(tf.keras.Model):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size 
        self.key = tf.keras.layers.Dense(head_size, use_bias = False)
        self.query = tf.keras.layers.Dense(head_size, use_bias = False)
        self.value = tf.keras.layers.Dense(head_size, use_bias = False)
    
    def call(self, x):
        #pega as dimensoes de batch, context size e channels do input 
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        trig = tf.linalg.band_part(tf.ones(shape = (T, T)), -1, 0)

        att_w = q @ tf.transpose(k, perm = [0, 2, 1]) *(k.shape[-1])**-0.5

        att_w = tf.where(trig == 0, -np.inf, att_w)

        att_w = tf.nn.softmax(att_w, axis = -1)
        
        output = att_w @ v

        return output




In [19]:
head = AttentionHead(head_size = 8)
ex = head(tf.random.normal(shape = (16, 3, 3)))
print(ex)

tf.Tensor(
[[[-5.36200583e-01  1.21979582e+00 -6.35650992e-01 -1.20261455e+00
    8.16121578e-01 -5.90881169e-01 -1.88537753e+00  9.94907081e-01]
  [-4.16900247e-01  7.61421621e-01 -3.38216722e-01 -7.95124292e-01
    4.66019571e-01 -3.30589116e-01 -1.42362368e+00  5.14793396e-01]
  [-3.21003705e-01  2.11422876e-01 -1.28724486e-01 -3.26458126e-01
    3.86039913e-02 -1.88218519e-01 -7.82809615e-01  5.95069975e-02]]

 [[-6.04891956e-01 -2.48096824e-01 -6.80191159e-01 -1.02991946e-01
   -4.14803743e-01 -9.44959819e-01 -9.54205617e-02  1.98871464e-01]
  [-3.58393669e-01 -3.85810018e-01  6.25850737e-01  9.35708582e-02
   -5.41782141e-01  4.84084487e-01 -1.04993725e+00 -1.04597259e+00]
  [-7.77639627e-01 -1.88245445e-01 -1.80304170e-01 -2.53557980e-01
   -4.93666917e-01 -4.53448623e-01 -1.16626990e+00 -3.70242864e-01]]

 [[-3.50939691e-01  4.10281539e-01 -3.12575102e-01 -4.90305573e-01
    2.05840632e-01 -3.52919459e-01 -8.68059814e-01  3.35443586e-01]
  [ 8.82045329e-02  5.15668631e-01  3.36

In [22]:
class MultiHeadAttention(tf.keras.Model):
    def __init__(self, head_size, n_heads):
        super().__init__()
        self.heads = [AttentionHead(head_size) for _ in range(n_heads)]
        self.proj = tf.keras.layers.Dense(512)
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, x):
        x = tf.concat([head(x) for head in self.heads], axis = -1)
        output = self.dropout(self.proj(x))

        return output

In [20]:
mha = MultiHeadAttention(8, 8)
rt = mha(tf.random.normal(shape = (16, 8, 3)))
print(rt)

tf.Tensor(
[[[-6.93779513e-02  4.50072169e-01 -2.37427264e-01 ... -7.00727046e-01
   -5.67331553e-01  1.10494947e+00]
  [ 5.99587336e-02  1.57032624e-01 -1.07536420e-01 ...  3.63230295e-02
   -2.61703551e-01  5.28916538e-01]
  [-1.17090642e-01  1.58971816e-01 -1.34923970e-02 ... -9.02118564e-01
   -2.74580657e-01  3.34211767e-01]
  ...
  [-1.53336301e-01  2.37333924e-01  1.20426945e-01 ... -5.48764825e-01
   -4.67314780e-01  4.67073649e-01]
  [-9.02180523e-02 -4.44706351e-01 -2.11766198e-01 ... -3.12023729e-01
   -2.52129257e-01  7.88422406e-01]
  [-1.03407882e-01 -5.40178549e-03 -6.14366345e-02 ... -3.50704908e-01
   -1.95939377e-01  1.77538559e-01]]

 [[-2.07829267e-01 -1.61697045e-01 -9.49870721e-02 ... -1.96300030e-01
   -1.84997439e-01  7.07827628e-01]
  [-1.28499627e-01  2.43657351e-01 -7.57647604e-02 ... -7.92656913e-02
   -8.50715935e-02  5.34071386e-01]
  [-8.82061273e-02  2.80394346e-01 -1.34909198e-01 ... -1.09406523e-01
   -3.55033726e-02  3.45052183e-01]
  ...
  [-1.707226

In [9]:
class FullyC(tf.keras.Model):
    def __init__(self, n_embed):
        super().__init__()
        self.dense = tf.keras.Sequential([
            tf.keras.layers.Dense(n_embed*4),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dense(n_embed),
            tf.keras.layers.Dropout(0.2)
        ])

    def call(self, x):
        output = self.dense(x)
        return output

In [10]:
class Block(tf.keras.Model):
    def __init__(self, n_head, n_embed):
        super().__init__()
        self.head_size = n_embed // n_head
        self.attention = MultiHeadAttention(self.head_size, n_head)
        self.fc = FullyC(n_embed)
        self.lnorm1 = tf.keras.layers.LayerNormalization()
        self.lnorm2 = tf.keras.layers.LayerNormalization()

    def call(self, x):
        #comunicacao entre os tokens
        x = x + self.attention(self.lnorm1(x))
        #aplicar o resultado da comunicacao
        x = x + self.fc(self.lnorm2(x))

        return x    

        

In [23]:
block = Block(16, 512)
ajs = block(tf.random.normal(shape=(16, 32, 512)))
print(ajs)

InvalidArgumentError: Exception encountered when calling Block.call().

[1m{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [16,32,3] vs. [16,32,512] [Op:AddV2] name: [0m

Arguments received by Block.call():
  • x=tf.Tensor(shape=(16, 32, 3), dtype=float32)